In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Install necessary libraries
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -qU datasets tensorboard rouge-score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 26.5 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 47.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
tensorflow 2.17.1 requires tensorbo

In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Model ID
model_id = "tiiuae/falcon-7b"

# Quantization Config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# Load Model with 4-bit Precision
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quantization_config,
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA configuration to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters for verification
print(f"Trainable parameters: {model.print_trainable_parameters()}")

# Load Dataset
dataset_path = "Cynaptics/persona-chat"
dataset = load_dataset(dataset_path, split="train")

# Define functions for prompt generation and tokenization
def generate_prompt(data_point):
    return f"""
    Persona B: {', '.join(data_point['persona_b'])}

    Conversation:
    {data_point['dialogue']}
    <AI>: {data_point['reference']}
    """.strip()

def preprocess_data(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized = tokenizer(
        full_prompt,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return {
        "input_ids": tokenized["input_ids"].squeeze(0),
        "attention_mask": tokenized["attention_mask"].squeeze(0),
        "labels": tokenized["labels"].squeeze(0),
    }

# Apply preprocessing
processed_dataset = dataset.map(preprocess_data, batched=False)
processed_dataset.save_to_disk("processed_dataset")

# Limit dataset size for faster training
small_dataset = processed_dataset.select(range(500))

# Set use_cache to False
model.config.use_cache = False
model.gradient_checkpointing_enable = False

from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer

# Training Arguments
training_args = TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir="output_dir",
    save_strategy='epoch',
    report_to="tensorboard",
    gradient_accumulation_steps=4,
)

# Data Collator with reduced max_length
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)

# Trainer
trainer = Trainer(
    model=model,
    train_dataset=small_dataset,
    args=training_args,
    data_collator=data_collator,
)

# Train the Model
trainer.train()

# Save the trained model and tokenizer
save_directory = "fine_tuned_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

# Function to download saved weights
def download_weights(path):
    import shutil
    shutil.make_archive("fine_tuned_model", 'zip', path)
    print("Weights saved and ready for download as fine_tuned_model.zip.")

download_weights(save_directory)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.848600


Model and tokenizer saved to fine_tuned_model
Weights saved and ready for download as fine_tuned_model.zip.


In [7]:
from rouge_score import rouge_scorer  # Import RougeScorer
from nltk.translate.bleu_score import sentence_bleu
import torch

# Initialize scorer for ROUGE evaluation
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

# Define test samples for evaluation
test_samples = [
    {
        "persona": [
            "I am a 25-year-old software developer.",
            "I love solving puzzles and reading sci-fi novels.",
            "I have a pet cat named Luna."
        ],
        "context": "Hi there! I think we met at the book club last week. What's your favorite genre?",
        "reference": "I love science fiction novels. They're my favorite genre. What about you?"
    },
    {
        "persona": [
            "I am a 40-year-old history professor.",
            "I enjoy hiking and cooking in my free time.",
            "I recently started learning how to play the guitar."
        ],
        "context": "Hello! I heard you recently took up a new hobby. Can you tell me more about it?",
        "reference": "Yes, I recently started learning to play the guitar. It's been a fun and rewarding experience."
    },
]

# Function to evaluate the model
def evaluate_model(model, tokenizer, test_samples):
    total_bleu = 0
    total_rouge_l = 0

    for idx, sample in enumerate(test_samples):
        persona = "\n".join([f"Persona: {line}" for line in sample["persona"]])
        prompt = f"""
        {persona}

        Context: {sample['context']}

        Response: (Provide a detailed and informative answer about your preferences, hobbies, or opinions.)
        """

        # Tokenize input prompt
        input_ids = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).input_ids.to(model.device)

        # Generate model responses using beam search for multiple outputs
        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                max_new_tokens=50,
                num_beams=5,  # Using beam search to generate multiple candidates
                no_repeat_ngram_size=2,  # Avoid repeating n-grams
                top_p=0.9,
                temperature=0.7,
                early_stopping=True,
                use_cache=False  # Disable caching for better performance in some cases
            )

        # Decode response and reference
        response = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
        response = response[len(prompt):].strip()  # Remove the prompt part

        # Post-process the response to clean up unwanted characters or format
        response = response.replace("\n", " ").strip()  # Remove any newlines
        reference = sample["reference"]

        # Compute BLEU score
        reference_tokens = [reference.split()]
        response_tokens = response.split()
        bleu = sentence_bleu(reference_tokens, response_tokens)

        # Compute ROUGE-L score
        rouge_scores = scorer.score(reference, response)
        rouge_l = rouge_scores["rougeL"].fmeasure

        total_bleu += bleu
        total_rouge_l += rouge_l

        # Print results
        print(f"Test Sample {idx + 1}:")
        print(f"Prompt:\n{prompt.strip()}")
        print(f"Reference Response:\n{reference}")
        print(f"Model Response:\n{response}")
        print(f"BLEU Score: {bleu:.4f}")
        print(f"ROUGE-L Score: {rouge_l:.4f}")
        print("-" * 80)

    # Print average scores
    avg_bleu = total_bleu / len(test_samples)
    avg_rouge_l = total_rouge_l / len(test_samples)
    print(f"Average BLEU Score: {avg_bleu:.4f}")
    print(f"Average ROUGE-L Score: {avg_rouge_l:.4f}")

# Run evaluation
evaluate_model(model, tokenizer, test_samples)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mas

Test Sample 1:
Prompt:
Persona: I am a 25-year-old software developer.
Persona: I love solving puzzles and reading sci-fi novels.
Persona: I have a pet cat named Luna.

        Context: Hi there! I think we met at the book club last week. What's your favorite genre?

        Response: (Provide a detailed and informative answer about your preferences, hobbies, or opinions.)
Reference Response:
I love science fiction novels. They're my favorite genre. What about you?
Model Response:
return self.get_response(response)      def get_persona(self, response):         persona = {}         for key, value in response.items():             if key in PERSONAS:                 persona[key]
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
--------------------------------------------------------------------------------
Test Sample 2:
Prompt:
Persona: I am a 40-year-old history professor.
Persona: I enjoy hiking and cooking in my free time.
Persona: I recently started learning how to play the guitar.

        

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_